In [1]:
import tensorly as tl
import numpy as np
from numpy import linalg as la
from sympy import *

from tensorly.decomposition import parafac
from tensorly.decomposition import non_negative_parafac_hals

In [6]:
def check_rank(tensor, rank, non_neg=True, n=10, tol=0.001, p=False):
    if non_neg:
        for k in range(n):
            weights, factors = non_negative_parafac_hals(tensor, n_iter_max=1000000, rank=rank, init='random')
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                # print(full)
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    else:
        for k in range(n):
            weights, factors = parafac(tensor, n_iter_max=1000000, rank=rank)
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    return False

#generates random rank 3 tensors
def rank_tree():
    return (low_tensor() + low_tensor() + low_tensor()) 

#generates random rank 4+ (hopefully) tensors
def rank_for():
    return  (low_tensor() + low_tensor() + low_tensor() + low_tensor())


# generates rank 1, 2x2x3 tensors
def low_tensor():
    max = 40
    a = np.random.randint(1, max, size=3)
    b = np.random.randint(1, max, size=3)
    c = np.random.randint(1, max, size=3)
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(3, 3, 3)) * 0.1
    return (tens,a,b,c)

In [7]:
def check(t, p = False):
    p000 = t[0][0][0]
    p001 = t[1][0][0]
    p002 = t[2][0][0]
    p100 = t[0][1][0]
    p101 = t[1][1][0]
    p102 = t[2][1][0]
    p010 = t[0][0][1]
    p011 = t[1][0][1]
    p012 = t[2][0][1]
    p110 = t[0][1][1]
    p111 = t[1][1][1]
    p112 = t[2][1][1]
    p210 = t[0][2][1]
    p211 = t[1][1][2]
    p212 = t[2][2][1]
    p220 = t[0][2][2]
    p221 = t[1][2][2]
    p222 = t[2][2][2]
    p120 = t[0][1][2]
    p121 = t[1][1][2]
    p122 = t[2][1][2]
    p200 = t[0][2][0]
    p201 = t[1][2][0]
    p202 = t[2][2][0]
    p020 = t[0][0][2]
    p021 = t[1][0][2]
    p022 = t[2][0][2]
    

def check_req(p):
    d1 = Matrix(p[0])
    d2 = Matrix(p[1])
    d3 = Matrix(p[2])
    v1 = Matrix(p[:,:,0])
    v2 = Matrix(p[:,:,1])
    v3 = Matrix(p[:,:,2])
    h1 = Matrix(p[:,0])
    h2 = Matrix(p[:,1])
    h3 = Matrix(p[:,2])
    zero = Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
    d1a = d1.adjugate()
    d2a = d2.adjugate()
    d3a = d3.adjugate()
    v1a = v1.adjugate()
    v2a = v2.adjugate()
    v3a = v3.adjugate()
    h1a = h1.adjugate()
    h2a = h2.adjugate()
    h3a = h3.adjugate()
    a1 = d1 * d2a * d3 - d3* d2a * d1
    a2 = d2 * d3a * d1 - d1* d3a * d2
    a3 = d3 * d1a * d2 - d2* d1a * d3
    b1 = v1 * v2a * v3 - v3* v2a * v1 
    b2 = v2 * v3a * v1 - v1* v3a * v2 
    b3 = v3 * v1a * v2 - v2* v1a * v3 
    c1 = h1 * h2a * h3 - h3* h2a * h1
    c2 = h2 * h3a * h1 - h1* h3a * h2 
    c3 = h3 * h1a * h2 - h2* h1a * h3 
    #print(c1,c2,c3)
    #print(a,b,c)
    a = check(a1) and check(a2) and check(a3)
    b = check(b1) and check(b2) and check(b3)
    c = check(c1) and check(c2) and check(c3)
    return a or b or c
    
def check(a):
    return max(abs(a)) < 20


In [12]:
res = {True:0, False:0}
wrong = []
for i in range(100000):
    t1 = low_tensor()
    t2 = low_tensor()
    t3 = low_tensor()
    t4 = low_tensor()
    v = np.random.randint(0,100)
    t = t1[0] + t2[0] + t3[0] + t4[0]
    if v%2 == 0:
        t = np.random.randint(0,100000,size = (3,3,3))
    key = check_req(t)
    res[key] = res[key] + 1
    if key:
        print(t,v)
        wrong.append((t,t1,t2,t3,t4,v))
        print(check_rank(t,3))
print(res)

{True: 0, False: 100000}


In [10]:
print(wrong)

[(array([[[10850.,  8478.,  1423.],
        [ 3522.,  2802.,   613.],
        [ 8188.,  6510.,  1245.]],

       [[ 8762., 10014.,  3415.],
        [ 6810.,  7206.,  2941.],
        [14014., 15990.,  6000.]],

       [[ 9458.,  9502.,  2751.],
        [ 5714.,  5738.,  2165.],
        [12072., 12830.,  4415.]]]), (array([[[9724., 7480., 1122.],
        [1768., 1360.,  204.],
        [6630., 5100.,  765.]],

       [[1144.,  880.,  132.],
        [ 208.,  160.,   24.],
        [ 780.,  600.,   90.]],

       [[4004., 3080.,  462.],
        [ 728.,  560.,   84.],
        [2730., 2100.,  315.]]]), array([17,  2,  7]), array([22,  4, 15]), array([26, 20,  3])), (array([[[ 756.,  588.,  126.],
        [1404., 1092.,  234.],
        [ 864.,  672.,  144.]],

       [[ 378.,  294.,   63.],
        [ 702.,  546.,  117.],
        [ 432.,  336.,   72.]],

       [[ 504.,  392.,   84.],
        [ 936.,  728.,  156.],
        [ 576.,  448.,   96.]]]), array([6, 3, 4]), array([ 7, 13,  8]), array([1

In [21]:
[(array([[[10850.,  8478.,  1423.],
        [ 3522.,  2802.,   613.],
        [ 8188.,  6510.,  1245.]],

       [[ 8762., 10014.,  3415.],
        [ 6810.,  7206.,  2941.],
        [14014., 15990.,  6000.]],

       [[ 9458.,  9502.,  2751.],
        [ 5714.,  5738.,  2165.],
        [12072., 12830.,  4415.]]]), (array([[[9724., 7480., 1122.],
        [1768., 1360.,  204.],
        [6630., 5100.,  765.]],

       [[1144.,  880.,  132.],
        [ 208.,  160.,   24.],
        [ 780.,  600.,   90.]],

       [[4004., 3080.,  462.],
        [ 728.,  560.,   84.],
        [2730., 2100.,  315.]]]), array([17,  2,  7]), array([22,  4, 15]), array([26, 20,  3])), (array([[[ 756.,  588.,  126.],
        [1404., 1092.,  234.],
        [ 864.,  672.,  144.]],

       [[ 378.,  294.,   63.],
        [ 702.,  546.,  117.],
        [ 432.,  336.,   72.]],

       [[ 504.,  392.,   84.],
        [ 936.,  728.,  156.],
        [ 576.,  448.,   96.]]]), array([6, 3, 4]), array([ 7, 13,  8]), array([18, 14,  3])), (array([[[  162.,   234.,    63.],
        [   90.,   130.,    35.],
        [  252.,   364.,    98.]],

       [[ 4536.,  6552.,  1764.],
        [ 2520.,  3640.,   980.],
        [ 7056., 10192.,  2744.]],

       [[ 3078.,  4446.,  1197.],
        [ 1710.,  2470.,   665.],
        [ 4788.,  6916.,  1862.]]]), array([ 1, 28, 19]), array([ 9,  5, 14]), array([18, 26,  7])), (array([[[ 208.,  176.,  112.],
        [ 260.,  220.,  140.],
        [ 442.,  374.,  238.]],

       [[2704., 2288., 1456.],
        [3380., 2860., 1820.],
        [5746., 4862., 3094.]],

       [[1872., 1584., 1008.],
        [2340., 1980., 1260.],
        [3978., 3366., 2142.]]]), array([ 2, 26, 18]), array([ 8, 10, 17]), array([13, 11,  7])), 81), (array([[[ 7249.,  5386.,  3618.],
        [ 7249.,  5386.,  3618.],
        [ 2827.,  2263.,  2147.]],

       [[22238., 12968., 10346.],
        [22238., 12968., 10346.],
        [14118.,  7966.,  7794.]],

       [[33556., 18724., 11303.],
        [33556., 18724., 11303.],
        [18462.,  9401.,  7446.]]]), (array([[[ 216.,  108.,   90.],
        [ 216.,  108.,   90.],
        [ 408.,  204.,  170.]],

       [[2376., 1188.,  990.],
        [2376., 1188.,  990.],
        [4488., 2244., 1870.]],

       [[2700., 1350., 1125.],
        [2700., 1350., 1125.],
        [5100., 2550., 2125.]]]), array([ 2, 22, 25]), array([ 9,  9, 17]), array([12,  6,  5])), (array([[[ 1248.,   312.,   208.],
        [ 1248.,   312.,   208.],
        [  816.,   204.,   136.]],

       [[10608.,  2652.,  1768.],
        [10608.,  2652.,  1768.],
        [ 6936.,  1734.,  1156.]],

       [[13104.,  3276.,  2184.],
        [13104.,  3276.,  2184.],
        [ 8568.,  2142.,  1428.]]]), array([ 2, 17, 21]), array([26, 26, 17]), array([24,  6,  4])), (array([[[ 5589.,  3888.,  1458.],
        [ 5589.,  3888.,  1458.],
        [ 1449.,  1008.,   378.]],

       [[ 8694.,  6048.,  2268.],
        [ 8694.,  6048.,  2268.],
        [ 2254.,  1568.,   588.]],

       [[17388., 12096.,  4536.],
        [17388., 12096.,  4536.],
        [ 4508.,  3136.,  1176.]]]), array([ 9, 14, 28]), array([27, 27,  7]), array([23, 16,  6])), (array([[[ 196., 1078., 1862.],
        [ 196., 1078., 1862.],
        [ 154.,  847., 1463.]],

       [[ 560., 3080., 5320.],
        [ 560., 3080., 5320.],
        [ 440., 2420., 4180.]],

       [[ 364., 2002., 3458.],
        [ 364., 2002., 3458.],
        [ 286., 1573., 2717.]]]), array([ 7, 20, 13]), array([14, 14, 11]), array([ 2, 11, 19])), 25), (array([[[ 9518.,  9518.,  9938.],
        [10117., 10117.,  7453.],
        [10397., 10397., 10841.]],

       [[28332., 28332., 34612.],
        [39857., 39857., 39265.],
        [35063., 35063., 39855.]],

       [[ 9606.,  9606., 12626.],
        [27262., 27262., 28294.],
        [23368., 23368., 28316.]]]), (array([[[ 3480.,  3480.,  4872.],
        [ 1680.,  1680.,  2352.],
        [ 1800.,  1800.,  2520.]],

       [[16240., 16240., 22736.],
        [ 7840.,  7840., 10976.],
        [ 8400.,  8400., 11760.]],

       [[ 1160.,  1160.,  1624.],
        [  560.,   560.,   784.],
        [  600.,   600.,   840.]]]), array([ 6, 28,  2]), array([29, 14, 15]), array([20, 20, 28])), (array([[[ 2160.,  2160.,  3960.],
        [ 1440.,  1440.,  2640.],
        [ 3600.,  3600.,  6600.]],

       [[ 4176.,  4176.,  7656.],
        [ 2784.,  2784.,  5104.],
        [ 6960.,  6960., 12760.]],

       [[ 4176.,  4176.,  7656.],
        [ 2784.,  2784.,  5104.],
        [ 6960.,  6960., 12760.]]]), array([15, 29, 29]), array([12,  8, 20]), array([12, 12, 22])), (array([[[  116.,   116.,   116.],
        [  841.,   841.,   841.],
        [  551.,   551.,   551.]],

       [[ 2900.,  2900.,  2900.],
        [21025., 21025., 21025.],
        [13775., 13775., 13775.]],

       [[ 3016.,  3016.,  3016.],
        [21866., 21866., 21866.],
        [14326., 14326., 14326.]]]), array([ 1, 25, 26]), array([ 4, 29, 19]), array([29, 29, 29])), (array([[[3762., 3762.,  990.],
        [6156., 6156., 1620.],
        [4446., 4446., 1170.]],

       [[5016., 5016., 1320.],
        [8208., 8208., 2160.],
        [5928., 5928., 1560.]],

       [[1254., 1254.,  330.],
        [2052., 2052.,  540.],
        [1482., 1482.,  390.]]]), array([18, 24,  6]), array([11, 18, 13]), array([19, 19,  5])), 19), (array([[[ 9611.,  2173.,  5185.],
        [16112.,  4448., 12051.],
        [ 9611.,  2173.,  5185.]],

       [[18894.,  3434., 13314.],
        [26314.,  5602., 20617.],
        [18894.,  3434., 13314.]],

       [[28605.,  5827., 20367.],
        [41259.,  9816., 33236.],
        [28605.,  5827., 20367.]]]), (array([[[ 936.,  624.,  936.],
        [1224.,  816., 1224.],
        [ 936.,  624.,  936.]],

       [[1560., 1040., 1560.],
        [2040., 1360., 2040.],
        [1560., 1040., 1560.]],

       [[3120., 2080., 3120.],
        [4080., 2720., 4080.],
        [3120., 2080., 3120.]]]), array([ 3,  5, 10]), array([13, 17, 13]), array([24, 16, 24])), (array([[[  384.,    48.,   408.],
        [  464.,    58.,   493.],
        [  384.,    48.,   408.]],

       [[ 8064.,  1008.,  8568.],
        [ 9744.,  1218., 10353.],
        [ 8064.,  1008.,  8568.]],

       [[10752.,  1344., 11424.],
        [12992.,  1624., 13804.],
        [10752.,  1344., 11424.]]]), array([ 1, 21, 28]), array([24, 29, 24]), array([16,  2, 17])), (array([[[ 2691.,  1053.,  3393.],
        [ 7774.,  3042.,  9802.],
        [ 2691.,  1053.,  3393.]],

       [[ 2070.,   810.,  2610.],
        [ 5980.,  2340.,  7540.],
        [ 2070.,   810.,  2610.]],

       [[ 3933.,  1539.,  4959.],
        [11362.,  4446., 14326.],
        [ 3933.,  1539.,  4959.]]]), array([13, 10, 19]), array([ 9, 26,  9]), array([23,  9, 29])), (array([[[ 5600.,   448.,   448.],
        [ 6650.,   532.,   532.],
        [ 5600.,   448.,   448.]],

       [[ 7200.,   576.,   576.],
        [ 8550.,   684.,   684.],
        [ 7200.,   576.,   576.]],

       [[10800.,   864.,   864.],
        [12825.,  1026.,  1026.],
        [10800.,   864.,   864.]]]), array([14, 18, 27]), array([16, 19, 16]), array([25,  2,  2])), 39), (array([[[12132., 12132., 13442.],
        [ 5463.,  5463.,  1762.],
        [ 6585.,  6585.,  9813.]],

       [[12944., 12944.,  5744.],
        [13225., 13225.,  5170.],
        [ 8187.,  8187.,  3894.]],

       [[21761., 21761.,  8834.],
        [18535., 18535.,  7612.],
        [11578., 11578.,  5812.]]]), (array([[[ 672.,  672.,   42.],
        [ 768.,  768.,   48.],
        [ 624.,  624.,   39.]],

       [[3584., 3584.,  224.],
        [4096., 4096.,  256.],
        [3328., 3328.,  208.]],

       [[2240., 2240.,  140.],
        [2560., 2560.,  160.],
        [2080., 2080.,  130.]]]), array([ 3, 16, 10]), array([14, 16, 13]), array([16, 16,  1])), (array([[[  648.,   648.,   432.],
        [ 1215.,  1215.,   810.],
        [  405.,   405.,   270.]],

       [[ 3672.,  3672.,  2448.],
        [ 6885.,  6885.,  4590.],
        [ 2295.,  2295.,  1530.]],

       [[ 5400.,  5400.,  3600.],
        [10125., 10125.,  6750.],
        [ 3375.,  3375.,  2250.]]]), array([ 3, 17, 25]), array([ 8, 15,  5]), array([27, 27, 18])), (array([[[ 7452.,  7452.,   648.],
        [ 3312.,  3312.,   288.],
        [ 3036.,  3036.,   264.]],

       [[ 4968.,  4968.,   432.],
        [ 2208.,  2208.,   192.],
        [ 2024.,  2024.,   176.]],

       [[13041., 13041.,  1134.],
        [ 5796.,  5796.,   504.],
        [ 5313.,  5313.,   462.]]]), array([12,  8, 21]), array([27, 12, 11]), array([23, 23,  2])), (array([[[ 3360.,  3360., 12320.],
        [  168.,   168.,   616.],
        [ 2520.,  2520.,  9240.]],

       [[  720.,   720.,  2640.],
        [   36.,    36.,   132.],
        [  540.,   540.,  1980.]],

       [[ 1080.,  1080.,  3960.],
        [   54.,    54.,   198.],
        [  810.,   810.,  2970.]]]), array([28,  6,  9]), array([20,  1, 15]), array([ 6,  6, 22])), 55)]

In [21]:
t = [[[10850.,  8478.,  1423.],
        [ 3522.,  2802.,   613.],
        [ 8188.,  6510.,  1245.]],

       [[ 8762., 10014.,  3415.],
        [ 6810.,  7206.,  2941.],
        [14014., 15990.,  6000.]],

       [[ 9458.,  9502.,  2751.],
        [ 5714.,  5738.,  2165.],
        [12072., 12830.,  4415.]]]
t = tl.tensor(t)

In [29]:
det(Matrix(t[:,:,0]))

0

In [28]:
det(Matrix(t[:,:,1]))

0

In [27]:
det(Matrix(t[:,:,2]))

0

In [20]:
t[0]

[[10850.0, 8478.0, 1423.0], [3522.0, 2802.0, 613.0], [8188.0, 6510.0, 1245.0]]